## Langchain
LangChain is a framework for developing applications powered by language models. It enables applications that are:

Data-aware: connect a language model to other sources of data

Agentic: allow a language model to interact with its environment

In [ ]:
pip install langchain

In [ ]:
pip install openai

In [2]:
import os
os.environ["OPENAI_API_KEY"]="..."

In [13]:
from langchain.llms import OpenAI

llm = OpenAI(temperature=0.9)

In [14]:
llm.predict("What is a good company name for a company that makes Indian bridal wears for mens?")

Retrying langchain.llms.openai.completion_with_retry.<locals>._completion_with_retry in 4.0 seconds as it raised APIConnectionError: Error communicating with OpenAI: ('Connection aborted.', ConnectionResetError(10054, 'An existing connection was forcibly closed by the remote host', None, 10054, None)).


'\n\nDulha Fashions.'

In [15]:
from langchain.chat_models import ChatOpenAI
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)

chat = ChatOpenAI(temperature=0)
chat.predict_messages([HumanMessage(content="Translate this sentence from English to Hindi. I love watching movies and sports on TV.")])


AIMessage(content='मुझे टीवी पर फिल्में और खेल देखना पसंद है।', additional_kwargs={}, example=False)

In [16]:
chat.predict("Translate this sentence from English to Hindi. I love watching movies and sports on TV.")


'मुझे टीवी पर फिल्में और खेल देखना पसंद है।'

# Prompt templates
LLM add the user input to a larger piece of text, called a prompt template, that provides additional context on the specific task at hand.

In [17]:
chat.predict('What is a good team name for cricket franchise of USA Premier League in Californai? Name should be close to Indians')

'California Indians'

In [18]:
from langchain.prompts import PromptTemplate

prompt = PromptTemplate.from_template("What is a good team name for cricket franchise of USA Premier League {city}? Name should be close to Indians")
prompt.format(city="California")

'What is a good team name for cricket franchise of USA Premier League California? Name should be close to Indians'

In [19]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)
chain.run("California")

'\n\nCalifornia Kings'

# Chains
Chains allow us to combine multiple components together to create a single, coherent application.

In [9]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt)


In [10]:
chain.run("California")

'\n\nGolden Warriors of California.'

In [20]:
prompt = PromptTemplate(
    input_variables=["product", "category"],
    template="What is a good company name for a company that makes {product} for {category}?",
)
chain = LLMChain(llm=llm, prompt=prompt)
print(chain.run({
    'product': "Indian bridal wears",
    'category': "womens"
    }))



Sari Couture Designs.


# Agents
To handle complex workflows, we need to be able to dynamically choose actions based on inputs. Agents help you to do this.

In [21]:
pip install google-search-results

Note: you may need to restart the kernel to use updated packages.


In [ ]:
os.environ["SERPAPI_API_KEY"] = "..."

In [22]:
from langchain.agents import AgentType, initialize_agent, load_tools
from langchain.llms import OpenAI

# The language model we're going to use to control the agent.
llm = OpenAI(temperature=0)

# The tools we'll give the Agent access to. Note that the 'llm-math' tool uses an LLM, so we need to pass that in.
tools = load_tools(["serpapi", "llm-math"], llm=llm)

# Finally, let's initialize an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

In [23]:
# Let's test it out!
agent.run("What was the high temperature in Mumbai yesterday in Fahrenheit? What is that number raised to the .023 power?")



> Entering new AgentExecutor chain...
 I need to find the temperature in Fahrenheit and then use a calculator to raise it to the .023 power.
Action: Search
Action Input: "High temperature in Mumbai yesterday in Fahrenheit"
Observation: Mumbai Weather History for the Previous 24 Hours ; 82 °F · 81 °F · 79 °F ...
Thought: I now need to use a calculator to raise 82 to the .023 power
Action: Calculator
Action Input: 82^.023
Observation: Answer: 1.1066689335946365
Thought: I now know the final answer
Final Answer: 1.1066689335946365

> Finished chain.


'1.1066689335946365'

In [24]:
agent.run("How many wickets Ravichandran Ashwin took against West Indies in current test match? Is it prime number?")



> Entering new AgentExecutor chain...
 I need to find out the number of wickets taken by Ashwin
Action: Search
Action Input: "Ravichandran Ashwin wickets West Indies test match"
Observation: Beside 5-60 from the first innings, Ashwin's dozen wickets in the match gave him an eighth 10-wicket haul in a test, tying Anil Kumble's India record. They are tied for fifth on the all-time list. Jadeja took 2-38 on Friday for a match haul of five wickets.
Thought: I need to find out if the number of wickets is a prime number
Action: Calculator
Action Input: 12
Observation: Answer: 12
Thought: I now know the final answer
Final Answer: Ravichandran Ashwin took 12 wickets against West Indies in the current test match, and 12 is not a prime number.

> Finished chain.


'Ravichandran Ashwin took 12 wickets against West Indies in the current test match, and 12 is not a prime number.'

# Memory
The chains and agents are stateless, but for many applications, it's necessary to reference past interactions. The Memory module gives you a way to maintain the application state. 
For example, chatbots, where you want to understand new messages in the context of past messages.

In [25]:
from langchain import OpenAI, ConversationChain

llm = OpenAI(temperature=0)
conversation = ConversationChain(llm=llm, verbose=True)

conversation.run("My name is Sanjay, I am currently staying Pune. I have done BE in Computer")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: My name is Sanjay, I am currently staying Pune. I have done BE in Computer
AI:

> Finished chain.


" Hi Sanjay, it's nice to meet you. I'm an AI and I'm programmed to answer questions about Pune. Can I help you with anything?"

In [26]:
conversation.run("Do you mind telling me full form of AI?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My name is Sanjay, I am currently staying Pune. I have done BE in Computer
AI:  Hi Sanjay, it's nice to meet you. I'm an AI and I'm programmed to answer questions about Pune. Can I help you with anything?
Human: Do you mind telling me full form of AI?
AI:

> Finished chain.


' Sure, AI stands for Artificial Intelligence. It is a branch of computer science that focuses on creating intelligent machines that can think and act like humans.'

In [27]:
conversation.run("What is India capital?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My name is Sanjay, I am currently staying Pune. I have done BE in Computer
AI:  Hi Sanjay, it's nice to meet you. I'm an AI and I'm programmed to answer questions about Pune. Can I help you with anything?
Human: Do you mind telling me full form of AI?
AI:  Sure, AI stands for Artificial Intelligence. It is a branch of computer science that focuses on creating intelligent machines that can think and act like humans.
Human: What is India capital?
AI:

> Finished chain.


" India's capital is New Delhi."

In [28]:
conversation.run("Who is USA president?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My name is Sanjay, I am currently staying Pune. I have done BE in Computer
AI:  Hi Sanjay, it's nice to meet you. I'm an AI and I'm programmed to answer questions about Pune. Can I help you with anything?
Human: Do you mind telling me full form of AI?
AI:  Sure, AI stands for Artificial Intelligence. It is a branch of computer science that focuses on creating intelligent machines that can think and act like humans.
Human: What is India capital?
AI:  India's capital is New Delhi.
Human: Who is USA president?
AI:

> Finished chain.


'  The current President of the United States is Joe Biden.'

In [29]:
conversation.run("Have I done my BE in Electonics?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: My name is Sanjay, I am currently staying Pune. I have done BE in Computer
AI:  Hi Sanjay, it's nice to meet you. I'm an AI and I'm programmed to answer questions about Pune. Can I help you with anything?
Human: Do you mind telling me full form of AI?
AI:  Sure, AI stands for Artificial Intelligence. It is a branch of computer science that focuses on creating intelligent machines that can think and act like humans.
Human: What is India capital?
AI:  India's capital is New Delhi.
Human: Who is USA president?
AI:   The current President of the United States is Joe Biden.
Human: Have I done my BE in Electonics?
AI:

> Finished chain.


' No, you said you have done BE in Computer.'